<a href="https://colab.research.google.com/github/kejeon/in_dev_RN20Q/blob/main/2_WC_train_resent20_q_without_KD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL = not IN_COLAB
USE_GITHUB = True
USE_DRIVE = False
device = 'cuda'

assert not (USE_GITHUB and USE_DRIVE)

In [2]:
if IN_COLAB:
  !pip install wandb -qU
  from google.colab import runtime
  if USE_GITHUB:
    !git clone https://github.com/kejeon/in_dev_RN20Q.git
    %cd '/content/in_dev_RN20Q'
  elif USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/GitHub/in_dev_RN20Q'

import wandb
import torch
from model.resnet4c10q import ResNet20_Q
from model.resnet4c10 import resnet20
from mylib.WCResNetTrainer import ResNetTrainer

wandb.login(key='e0c11d3ff2bee4c8775ba05863038fdac671c043')

fatal: destination path 'in_dev_RN20Q' already exists and is not an empty directory.
/content/in_dev_RN20Q


wandb: Currently logged in as: jke1994. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
lr = 0.1
arch_tag = "ResNet20_Q"
batch_size = 128
dataset = "CIFAR10"
eta = 100
a_bit = 4
w_bit = 4

student_model = ResNet20_Q(a_bit=a_bit, w_bit=w_bit)

run = wandb.init(
    # set the wandb project where this run will be logged
    project="ResNet20_WC",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "eta": eta,
    "a_bit": a_bit,
    "w_bit": w_bit,
    "architecture": arch_tag,
    "batch_size": batch_size,
    "dataset": dataset,
    "style": 'without_KD'
    }
)

my_trainer = ResNetTrainer(dataset=dataset,
                           arch_tag=arch_tag,
                           eta=eta,
                           model=student_model,
                           device = device,
                           batch_size=batch_size,
                           lr=lr)

my_trainer.train_script(200)

wandb.finish()

runtime.unassign()

lr,█▆▁
test_acc,█▁
test_loss,▁█
train_acc,▁▅█
train_loss,█▄▁
wc_loss,█▄▁
xent_loss,█▁▂
lr,0.09996
test_acc,0.3382
test_loss,0.01596
train_acc,0.46516


Files already downloaded and verified
Files already downloaded and verified

Epoch: 0


Epoch 0: 100%|██████████| 391/391 [00:20<00:00, 18.81it/s, train_acc=0.285, xentropy=1.951]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:814: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


test_acc: 0.331
Saving..

Epoch: 1


Epoch 1: 100%|██████████| 391/391 [00:20<00:00, 19.53it/s, train_acc=0.408, xentropy=1.601]


...

Epoch 6: 100%|██████████| 391/391 [00:20<00:00, 19.14it/s, train_acc=0.691, xentropy=0.888]


test_acc: 0.646
Saving..

Epoch: 7


Epoch 7: 100%|██████████| 391/391 [00:20<00:00, 18.72it/s, train_acc=0.713, xentropy=0.831]


test_acc: 0.665
Saving..

Epoch: 8


Epoch 8:  58%|█████▊    | 225/391 [00:11<00:08, 20.52it/s, train_acc=0.723, xentropy=0.801]

In [3]:
lr = 0.1
arch_tag = "ResNet20_Q"
batch_size = 128
dataset = "CIFAR10"
eta = 0.1
a_bit = 4
w_bit = 4

student_model = ResNet20_Q(a_bit=a_bit, w_bit=w_bit)

In [4]:
from mylib.KL_div import avg_kl_div_loss


In [6]:
print(avg_kl_div_loss(student_model))

0


In [7]:
import torch
import torch.nn as nn
from model.q_lib import Conv2d_Q

def kl_div_loss(conv_filter):
    num_kernel = conv_filter.shape[0]
    mean_vec = torch.mean(conv_filter, dim=(1,2,3))
    std_vec = torch.std(conv_filter, dim=(1,2,3))

    mean_mat = mean_vec.repeat(num_kernel,1)
    std_mat = std_vec.repeat(num_kernel,1)

    mean_mat_T = mean_mat.T
    std_mat_T = std_mat.T

    kl_div_mat = torch.div(std_mat.T, std_mat).log() + (
        std_mat.square() + (mean_mat - mean_mat.T).square())/(2*std_mat.T.square()) - 1/2

    kl_div = torch.mean(kl_div_mat) * 2/(num_kernel*(num_kernel-1))

    return kl_div



In [32]:
def avg_kl_div_loss(model):

    def nn_traversal(root, kl_div_total):
        for name, layer in root.named_children():
            if isinstance(layer, Conv2d_Q):
                if layer.in_channels != layer.out_channels:
                    continue
                my_tensor = layer.weight
                kl_div_total += kl_div_loss(my_tensor)
            else:
                if len(layer._modules) == 0:
                    continue
                kl_div_total=nn_traversal(layer, kl_div_total)
        return kl_div_total

    kl_div_total=nn_traversal(model, 0)

    return kl_div_total

In [29]:
student_model.layers_2n[0].conv1.out_channels

16